# Check if people are wearing masks on a given image

In [2]:
import numpy as np
import  os 
import cv2
# import matplotlib.pyplot as plt

from mtcnn import MTCNN

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model


Using TensorFlow backend.


In [3]:
# Initiate the MTCNN face detector 
detector = MTCNN()

In [4]:
# Load mask detection model
mask_detect_model = load_model('saved_model/my_model_v3.h5')

In [7]:
def check_mask(img):
    '''
        :param: img -> It can be a path to an image or a variable that contains an image
         
         Detects and extract faces, then passes the faces to our mask detection model to decide
         they have masks or not. Finally, it draws bounding boxes on the images with decided labels on top
         
    '''
    # Look for a string path to the image file
    if isinstance(img,str):
        # Load the image from its path
        image_original = cv2.imread(img)
        # copy the image
        image = image_original[:]
    else:
        image = img[:]
        
        
    detection_results = detector.detect_faces(image)
    
    for i in range(len(detection_results)):
        confidence = detection_results[i]['confidence']
        # Check if the model is more than 50% confident about finding a face
        if confidence > 0.5:
            # Get the coordinates of the bounding box(location of the face)
            x,y,w,h = detection_results[i]['box']
            
            # Slice the face from the image and preprocess it
            # pass it to the mask detection model 
            face = image[y:y+h, x:x+w]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (160,160))
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)
            
            # Put the extracted face to mask detection model 
            # to check for a mask
            (no_mask, mask) = mask_detect_model.predict(face)[0]
            
            # Choose the label and the color of the bounding box
            # according to the prediction
            label = ''
            color = (0,0,0)
            if mask > no_mask:
                label = 'MASK'
                color = (0,255,0)
            else:
                label = 'NO MASK'
                color = (0,0,255)
            label = "{}: {:.2f}%".format(label, max(mask, no_mask) * 100)
            
            # Put the label and the accuracy on top of the face
            cv2.putText(image, label, (x, y-7),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(image, (x,y),(x+w, y+h), color, 2)
    cv2.imshow("Output", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [6]:
check_mask('test_images/tribun.jpg')


In [ ]:
# if cv2 window gets stuck, run this 
cv2.destroyAllWindows()
cv2.release()